# This example demonstrate how to train region classification (standart image classification model).

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

info = modelhub.download_dataset_for_model("numberplate_options")
PATH_TO_DATASET = info["dataset_path"]

# or local path dataset
# PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [4]:
PATH_TO_DATASET

'/mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./dataset/OptionsDetector/numberplate_options/autoriaNumberplateOptionsDataset-2021-05-17'

In [5]:
DATASET_NAME = "options"
VERSION = "2021_07_08_pytorch_lightning"

RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [6]:
from NomeroffNet import OptionsDetector

# definde your parameters
class MyNpClassificator(OptionsDetector):
    def __init__(self):
        OptionsDetector.__init__(self)
        
        # outputs 2
        self.class_region = [
            "xx-unknown", 
            "eu-ua-2015", 
            "eu-ua-2004", 
            "eu-ua-1995", 
            "eu", 
            "xx-transit", 
            "ru", 
            "kz", 
            "eu-ua-ordlo-dnr", 
            "eu-ua-ordlo-lnr", 
            "ge", 
            "by", 
            "su", 
            "kg"
        ]
        
        # output 3
        self.count_lines = ["0", "1", "2", "3"]
        
        self.epochs           = 100
        self.batch_size       = 64
        
        # count gpu for train
        self.gpus = 1
        
        self.height         = 64
        self.width         = 295


In [7]:
# initialize region detector.
npClassificator = MyNpClassificator()
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

START PREPARING
DATA PREPARED


In [13]:
# train
model = npClassificator.train()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name            | Type        | Params
-------------------------------------------------
0  | inp_conv        | Conv2d      | 896   
1  | conv1           | Conv2d      | 18.5 K
2  | conv2           | Conv2d      | 73.9 K
3  | conv3           | Conv2d      | 147 K 
4  | pool            | MaxPool2d   | 0     
5  | dropout_reg     | Dropout     | 0     
6  | fc1_reg         | Linear      | 2.1 M 
7  | fc2_reg         | Linear      | 131 K 
8  | batch_norm_reg  | BatchNorm1d | 1.0 K 
9  | fc3_reg         | Linear      | 3.6 K 
10 | dropout_line    | Dropout     | 0     
11 | fc1_line        | Linear      | 2.1 M 
12 | fc2_line        | Linear      | 131 K 
13 | batch_norm_line | BatchNorm1d | 1.0 K 
14 | fc3_line        | Linear      | 1.0 K 
-------------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



[INFO] best model path /mnt/store/nomeroff-net/nomeroff-net/train/lightning_logs/version_3/checkpoints/epoch=86-step=36800.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.989561140537262, 'test_loss': 1.2573670148849487}
--------------------------------------------------------------------------------


In [14]:
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

START PREPARING
DATA PREPARED


In [8]:
npClassificator.test()

AttributeError: 'NoneType' object has no attribute 'test'

In [16]:
npClassificator.save(RESULT_PATH)

model save to /mnt/store/nomeroff-net/nomeroff-net/./data/models/numberplate_options_2021_07_08_pytorch_lightning.ckpt


# Classification Report

In [10]:
npClassificator.load(RESULT_PATH)

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=14, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(

In [15]:
from NomeroffNet.data_modules.option_img_generator import ImgGenerator
from sklearn.metrics import classification_report
import torch

def options_classification_report(npClassificator, 
                                  path_dir=os.path.join(PATH_TO_DATASET, "test"), 
                                  mode_torch="gpu"):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    imageGenerator.build_data()
    gen = imageGenerator.path_generator()
    
    out_idx_reg_all = []
    out_idx_line_all = []
    label_idx_reg_all = []
    label_idx_line_all = []
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
            npClassificator.model.cuda()
        with torch.no_grad():
            outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        out_idx_reg_all.extend(out_idx_reg)
        out_idx_line_all.extend(out_idx_line)
        label_idx_reg_all.extend(label_idx_reg)
        label_idx_line_all.extend(label_idx_line)
    #print("out_idx_reg_all", out_idx_reg_all)
    print("\n\nclass region classification report:")
    print(classification_report(label_idx_reg_all, 
                                out_idx_reg_all, 
                                target_names=npClassificator.class_region))
    print("\n\ncount lines classification report:")
    print(classification_report(label_idx_line_all, 
                                out_idx_line_all, 
                                target_names=npClassificator.count_lines))

In [16]:
options_classification_report(npClassificator)



class region classification report:
                 precision    recall  f1-score   support

     xx-unknown       1.00      0.03      0.06      1303
     eu-ua-2015       0.40      1.00      0.57      1105
     eu-ua-2004       0.76      0.99      0.86      1057
     eu-ua-1995       1.00      0.85      0.92       916
             eu       0.77      0.88      0.82       999
     xx-transit       1.00      0.15      0.26       410
             ru       1.00      0.34      0.50       190
             kz       0.97      0.77      0.86       666
eu-ua-ordlo-dnr       0.46      1.00      0.63        48
eu-ua-ordlo-lnr       0.44      1.00      0.61        30
             ge       0.96      0.14      0.24       375
             by       0.88      0.91      0.89        23
             su       0.71      0.96      0.81       271
             kg       0.99      0.86      0.92        95

       accuracy                           0.66      7488
      macro avg       0.81      0.71      0.64  

# Check small accuracy images

In [12]:
npClassificator.load(RESULT_PATH)

NPOptionsNet(
  (inp_conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout_reg): Dropout(p=0.2, inplace=False)
  (fc1_reg): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_reg): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_reg): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_reg): Linear(in_features=256, out_features=15, bias=True)
  (dropout_line): Dropout(p=0.2, inplace=False)
  (fc1_line): Linear(in_features=4096, out_features=512, bias=True)
  (fc2_line): Linear(in_features=512, out_features=256, bias=True)
  (batch_norm_line): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3_line): Linear(

In [13]:
%matplotlib inline

In [20]:
from NomeroffNet.data_modules.option_img_generator import ImgGenerator
import torch
import json
import cv2
import matplotlib.pyplot as plt

def show_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

def small_acc(npClassificator, 
              path_dir=os.path.join(PATH_TO_DATASET, "test"), 
              mode_torch="gpu", 
              min_acc=0.75):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    print("start imageGenerator build")
    imageGenerator.build_data()
    print("end imageGenerator build")
    gen = imageGenerator.path_generator()
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
        outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        # check reg
        for o, l, p, tens in zip(out_idx_reg, label_idx_reg, img_paths, outputs[0]):
            if o != l:
                print("[WRONG REGION PREDICTED] pred", o, l, p)
                show_img(p)
            elif tens[l] < min_acc:
                print("[SMALL ACC REGION PREDICTED]  pred", o, l, tens[l], p)
                show_img(p)
        # check line
#         for o, l, p, tens in zip(out_idx_line, label_idx_line, img_paths, outputs[1]):
#             if o != l:
#                 print("[WRONG LINE PREDICTED]  pred", o, l, p)
#                 show_img(p)
#             elif tens[l] < min_acc:
#                 print("[SMALL ACC LINE PREDICTED]  pred", o, l, tens[l], p)
#                 show_img(p)

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "test"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "train"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "val"))